In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/seg_train/seg_train'
test_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/seg_test/seg_test'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.1, zoom_range = 0.15, horizontal_flip= True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size = (150, 150),
                                                    batch_size = 16,
                                                    color_mode = 'rgb',
                                                    class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size = (150, 150),
                                                  batch_size = 16,
                                                  color_mode = 'rgb',
                                                  class_mode = 'categorical')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [ ]:
baseModel = InceptionV3(weights = 'imagenet', include_top = False, input_shape = (150, 150, 3))

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
inception = baseModel.output
inception = GlobalAveragePooling2D()(inception)
inception = Dropout(0.2)(inception)
predictions = Dense(6, activation = 'softmax')(inception)

In [ ]:
InceptionV3_model = Model(inputs = baseModel.input, outputs = predictions)

In [ ]:
adam = Adam(learning_rate = 0.002)

In [ ]:
InceptionV3_model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
InceptionV3_model.fit(train_generator, batch_size = 16, epochs = 5, validation_data = test_generator)

Epoch 1/5
878/878 [==============================] - 7074s 8s/step - loss: 1.2201 - accuracy: 0.5428 - val_loss: 1.5553 - val_accuracy: 0.5753
Epoch 2/5
878/878 [==============================] - 297s 338ms/step - loss: 0.9268 - accuracy: 0.6729 - val_loss: 1.6604 - val_accuracy: 0.6647
Epoch 3/5
333/878 [==========>...................] - ETA: 2:53 - loss: 0.8396 - accuracy: 0.7074